In [ ]:
import pandas as pd
from google_sheets import get_spreadsheet_data, update_column_value, delete_column

DOCUMENT_NAME = "Job_Application_Tracker_Copy"

## Fetch the data from the datasheet

In [ ]:
df = get_spreadsheet_data(DOCUMENT_NAME)
easy_apply = df[df['Comments'].str.contains('LinkedIn Easy Apply', na=False)]
active_applications = df[df["Offer"] != "No"]

In [ ]:
# Applications needing follow-up with differentiated thresholds
# - 5 days for applications WITH last communication date
# - 14 days for applications using application date as fallback

# Find the Application Date column (name includes line break from spreadsheet)
app_date_col = [col for col in df.columns if col.startswith("Application Date")][0]

# Filter applications where Offer is empty (no response yet)
pending = df[df["Offer"].isna() | (df["Offer"] == "")].copy()

# Parse dates
pending['Last Communication Date'] = pd.to_datetime(pending['Last Communication Date'], errors='coerce')
pending[app_date_col] = pd.to_datetime(pending[app_date_col], errors='coerce')

# Flag rows where Last Communication Date is missing (will use Application Date)
pending['Used_Application_Date'] = pending['Last Communication Date'].isna()

# Use Last Communication Date, falling back to Application Date
pending['Effective Date'] = pending['Last Communication Date'].fillna(pending[app_date_col])

# Calculate days since last contact
pending['Days Since Contact'] = (pd.Timestamp.now() - pending['Effective Date']).dt.days

# Apply differentiated thresholds based on whether we have a last communication date
# - 5 days if we have Last Communication Date
# - 14 days if using Application Date as fallback
needs_followup = pending[
    (
        ((~pending['Used_Application_Date']) & (pending['Days Since Contact'] > 7)) |
        ((pending['Used_Application_Date']) & (pending['Days Since Contact'] > 14))
    )
]

# Display formatted output
print(f"Applications needing follow-up: {len(needs_followup)}")
print(f"  - With last communication (>5 days): {len(needs_followup[~needs_followup['Used_Application_Date']])}")
print(f"  - Without last communication (>14 days): {len(needs_followup[needs_followup['Used_Application_Date']])}")
print("\n" + "=" * 80)

for _, row in needs_followup.sort_values('Days Since Contact', ascending=False).iterrows():
    date_source = "Application Date" if row['Used_Application_Date'] else "Last Communication"
    print(f"Company:      {row['Company']}")
    print(f"Role:         {row['Role Title']}")
    print(f"Days:         {row['Days Since Contact']} (based on {date_source})")
    comments = row['Comments'] if pd.notna(row['Comments']) and row['Comments'] != "" else "N/A"
    print(f"Comments:     {comments}")
    print("-" * 80)

In [ ]:
# Application Analytics

app_date_col = [col for col in df.columns if col.startswith("Application Date")][0]

analytics = df[df['Company'].notna() & (df['Company'] != "")].copy()
analytics['Last Communication Date'] = pd.to_datetime(analytics['Last Communication Date'], errors='coerce')
analytics[app_date_col] = pd.to_datetime(analytics[app_date_col], errors='coerce')

total_applications = len(analytics)

# --- No Answer ---
no_answer_count = analytics['Comments'].str.contains('No answer', na=False).sum()
no_answer_pct = no_answer_count / total_applications * 100

# --- Average Closure Time (Application Date → Last Communication Date for closed applications with answer) ---
closed = analytics[analytics['Offer'].notna() & (analytics['Offer'] != "")].copy()
closed = closed[~closed['Comments'].str.contains('No answer', na=False)]
closed['Closure Days'] = (closed['Last Communication Date'] - closed[app_date_col]).dt.days
# Only consider rows where both dates are available
closed_with_dates = closed[closed['Closure Days'].notna()]
avg_closure_days = closed_with_dates['Closure Days'].mean()
median_closure_days = closed_with_dates['Closure Days'].median()

# --- LinkedIn Easy Apply % ---
easy_apply_count = analytics['Comments'].str.contains('LinkedIn Easy Apply', na=False).sum()
easy_apply_pct = easy_apply_count / total_applications * 100

# --- LinkedIn Easy Apply and No Answer ---
linked_in_easy_apply_and_no_answer_count = (analytics['Comments'].str.contains('No answer', na=False) & analytics['Comments'].str.contains('LinkedIn Easy Apply', na=False)).sum()
no_answer_with_linkedin_pct = linked_in_easy_apply_and_no_answer_count / easy_apply_count * 100

had_interview_count = len(analytics[analytics["Interview Time, Date \n& Interviewer Name"].notna() & (analytics["Interview Time, Date \n& Interviewer Name"] != "")])
had_interview_pct = had_interview_count / total_applications * 100

# --- Display ---
print("APPLICATION ANALYTICS")
print("=" * 50)

print(f"\nTotal applications:          {total_applications}")

print(f"\n--- Response Status ---")
print(f"No answer:                   {no_answer_count} ({no_answer_pct:.1f}%)")
print(f"Got answer:                  {total_applications - no_answer_count} ({100 - no_answer_pct:.1f}%)")

print(f"\n--- Closure Time ---")
print(f"Closed applications with answer:         {len(closed)} ({len(closed) / total_applications * 100:.1f}%)")
print(f"Average closure time:        {avg_closure_days:.1f} days")
print(f"Median closure time:         {median_closure_days:.1f} days")

print(f"\n--- Application Method ---")
print(f"LinkedIn Easy Apply:         {easy_apply_count} ({easy_apply_pct:.1f}%)")
print(f"Other methods:               {total_applications - easy_apply_count} ({100 - easy_apply_pct:.1f}%)")

print(f"\n--- Response Status for LinkedIn Easy Apply---")
print(f"No answer:                   {linked_in_easy_apply_and_no_answer_count} ({no_answer_with_linkedin_pct:.1f}%)")
print(f"Got answer:                  {easy_apply_count - linked_in_easy_apply_and_no_answer_count} ({100 - no_answer_with_linkedin_pct:.1f}%)")

print(f"\n--- Had interview ---")
print(f"At least one interview:        {had_interview_count} ({had_interview_pct:.1f}%)")
print(f"No interview:                  {total_applications - had_interview_count} ({100 - had_interview_pct:.1f}%)")

In [ ]:
num_updated_columns = update_column_value(
    sheet_name=DOCUMENT_NAME,
    filter_column="Response \n(Drop Down List)",
    filter_value="Positive Email",
    target_column="Response \n(Drop Down List)",
    new_value="",
)

print(f"I have updated {num_updated_columns} rows")